In [2]:
import pandas as pd
import nltk

In [133]:
import numpy as np

### Read the list of ingredients and leave only the short version

In [197]:
ingredients=pd.read_csv('data/ingredients.csv')
ingredients.drop('Column1', axis=1, inplace=True)

In [205]:
ingredients.head()

,full name,stem,substitute 1,substitute 2,substitute 3,food type
0,double cream,cream,NaN,NaN,NaN,NaN
1,flax seeds,flaxseed,NaN,NaN,NaN,NaN
2,farro,farro,NaN,NaN,NaN,NaN
3,applesauce,applesauce,NaN,NaN,NaN,NaN
4,fresh cranberries,cranberry,NaN,NaN,NaN,NaN


In [244]:
ingredients_short=ingredients[['stem', 'substitute 1', 'substitute 2', 'substitute 3', 'food type']]

In [245]:
ingredients_short=ingredients_short.drop_duplicates('stem')

In [246]:
ingredients_short.shape

(1929, 5)

In [247]:
ingredients_short=ingredients_short.reset_index()
ingredients_short.drop('index', axis=1, inplace=True)

In [204]:
#ingredients_short=ingredients_short.replace(np.nan, 0)

In [248]:
ingredients_short.head()

,stem,substitute 1,substitute 2,substitute 3,food type
0,cream,NaN,NaN,NaN,NaN
1,flaxseed,NaN,NaN,NaN,NaN
2,farro,NaN,NaN,NaN,NaN
3,applesauce,NaN,NaN,NaN,NaN
4,cranberry,NaN,NaN,NaN,NaN


### Add substitutes

In [149]:
f=open("data/substitutes.txt", "r")
substitutes=f.read()
substitutes=eval(substitutes)
substitutes=[list(x) for x in substitutes]

In [150]:
substitutes[:2]

[['olive oil', 'canola oil'], ['olive oil', 'peanut oil']]

In [151]:
check=list(set([y for x in substitutes for y in x]))

In [152]:
[x for x in check if x not in list(ingredients.stem)]

['chicken broth',
 'agave',
 'vegetable broth',
 'scallion',
 'beef broth',
 'minced garlic',
 'savory',
 'leeks',
 'tamari sauce',
 'half and half',
 'chive',
 'poblano pepper']

In [153]:
replace={'chicken broth':'chicken stock', 'agave':'agave nectar', 'tamari sauce':'tamari soy sauce', 
 'scallion':'chives', 'vegetable broth':'vegetable stock', 'savory': 'summer savory', 'poblano pepper':'poblano chiles',
 'chive':'chives', 'cornmeal': 'corn meal', 'leeks':'leek', 'half and half':'half-and-half', 'beef broth': 'beef stock', 'minced garlic': 'garlic'
}

In [154]:
for i in range(len(substitutes)):
    y=set(replace.keys()).intersection(substitutes[i])
    if len(y)>0:
        y=list(y)
        for yy in y:
            substitutes[i]=[replace[yy] if xx==yy else xx for xx in substitutes[i]]

In [155]:
check=list(set([y for x in substitutes for y in x]))
[x for x in check if x not in list(ingredients.stem)]

[]

### Incorporate additional items into the dataset

In [294]:
#ingredients_short['food type']=ingredients['food type']

In [296]:
ingredients_short.head(10)

,stem,substitute 1,substitute 2,substitute 3,food type
0,cream,half-and-half,NaN,NaN,NaN
1,flaxseed,NaN,NaN,NaN,NaN
2,farro,NaN,NaN,NaN,NaN
3,applesauce,NaN,NaN,NaN,NaN
4,cranberry,NaN,NaN,NaN,NaN
5,schmaltz,pork fat,NaN,NaN,NaN
6,black tea,NaN,NaN,NaN,NaN
7,green beans,bean,NaN,NaN,NaN
8,dried orange peel,NaN,NaN,NaN,NaN
9,garlic salt,salt,NaN,NaN,NaN


In [ ]:
for x in substitutes:
    idx=ingredients_short[ingredients_short['stem']==x[0]].index
    if x[1] not in ingredients_short.loc[idx][['substitute 1', 'substitute 2', 'substitute 3']]:
        if type(list(ingredients_short.loc[idx, 'substitute 1'])[0])!=str:
            ingredients_short.set_value(idx, 'substitute 1', x[1])
        elif type(list(ingredients_short.loc[idx, 'substitute 2'])[0])!=str:
            ingredients_short.set_value(idx, 'substitute 2', x[1])        
        elif type(list(ingredients_short.loc[idx, 'substitute 3'])[0])!=str:
            ingredients_short.set_value(idx, 'substitute 3', x[1])
        
        else:
                                        print('add column')
                                        break
                                        

### Assign food type

In [277]:
meat=['lamb', 'beef', 'meat', 'mutton', 'veal', 'venison', 'pork',
      'turkey', 'bacon', 'ham', 'hot dogs', 'jamon', 'prosciutto', 'salami', 'sausage', 'rabbit']

poultry=['chicken', 'turkey', 'duck', 'grouse', 'pheasant']

fish=['fish', 'salmon', 'cod', 'tilapia', 'catfish', 'halibut', 'mahi', 'tuna', 'mackerel',  
      'swordfish', 'sole', 'bass', 'anchovy', 'herring', 'haddock', 'monkfish', 'mullet', 'sardines', 'swordfish', 
     'trout', 'caviar', 'turbot', 'snapper', 'sturgeon', 'flounder', 'barramundi']

seafood=['crabs', 'shells', 'shrimp', 'clam', 'lobster', 'octopus', 'scallops', 'squid', 'eel', 'snail']

dairy=['cheese', 'milk', 'mascarpone', 'butter' 'buttermilk', 'ice cream', 
       'sour cream', 'yogurt', 'yoghurt', 'kefir', 'custard', 'cream']

milk_exclude=['coconut', 'soymilk', 'soy', 'cashew', 'almond', 'rice']

egg=['egg', 'eggs', 'yolk', 'whites']

#vegetables
vegetables=list(pd.read_csv('data/vegetables.csv')['Vegetables'])
vegetables=[x.lower() for x in vegetables]
vegetables=[x.split('/') for x in vegetables]
vegetables=[y for x in vegetables for y in x]

# fruit
fruits=list(pd.read_csv('data/fruits.csv')['Fruits'])
fruits=[x.lower() for x in fruits]
fruits=[x.split('/') for x in fruits]
fruits=[y for x in fruits for y in x]


exclude=['oil', 'water','juice','salt','jam', 'brandy','cider','vinegar', 'concentrate','schnapps','nectar','liqueur', 'flour','milk', 'extract',
'compote','peel','cream','sorbet','gelatin','syrup','sauce','bread','flour','starch','tortillas','ale','beer','salse']

d=dict(zip(['meat', 'poultry', 'fish', 'seafood', 'egg', 'vegetable', 'fruit', 'dairy'], 
     [red_meat, poultry, fish, seafood, egg, vegetables, fruits]))

In [269]:
d['red_meat']

['lamb',
 'beef',
 'meat',
 'mutton',
 'veal',
 'venison',
 'pork',
 'turkey',
 'bacon',
 'ham',
 'hot dogs',
 'jamon',
 'prosciutto',
 'salami',
 'sausage',
 'rabbit']

In [298]:
for i in ingredients_short.index:
    if type(ingredients_short.loc[i]['food type'])!=str:
        words=list(ingredients_short.loc[i][['stem','substitute 1']].dropna())
        words=[x.split() for x in words]
        words=[y for x in words for y in x]
        for key in d.keys():        
            if len(set(d[key]).intersection(words))>0:
                if key=='dairy':                    
                    if len(set(milk_exclude).intersection(words))==0:
                        ingredients_short.set_value(i, 'food type', key)  
                if key in ['fruit', 'vegetable']:                    
                    if len(set(exclude).intersection(words))==0:
                        ingredients_short.set_value(i, 'food type', key)  
                else:
                    ingredients_short.set_value(i, 'food type', key)   
                break

In [301]:
ingredients_short.head(10)

,stem,substitute 1,substitute 2,substitute 3,food type
0,cream,half-and-half,NaN,NaN,NaN
1,flaxseed,NaN,NaN,NaN,NaN
2,farro,NaN,NaN,NaN,NaN
3,applesauce,NaN,NaN,NaN,NaN
4,cranberry,NaN,NaN,NaN,fruit
5,schmaltz,pork fat,NaN,NaN,meat
6,black tea,NaN,NaN,NaN,NaN
7,green beans,bean,NaN,NaN,vegetable
8,dried orange peel,NaN,NaN,NaN,NaN
9,garlic salt,salt,NaN,NaN,NaN


In [302]:
ingredients_short.to_csv('ingredients_short.csv', encoding='utf-8')